In [151]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
# from models.vince_models import Net2B

In [152]:
# load the dataset
train_input,train_target, train_classes, test_input, test_target, test_classes = load()
train_target=train_target.float() # necessary ?
test_target=test_target.float()

In [215]:
# models
class Net2C(nn.Module):

    def __init__(self, nb_hidden):
        super(Net2C, self).__init__()
        
        # number of input channels is 2.
        self.conv1 = nn.Conv2d(2, 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 3)
        self.fc1 = nn.Linear(256, nb_hidden)
        #self.dropout = nn.Dropout()
        self.fc2 = nn.Linear(nb_hidden, 1) # single output
        self.sigmoid = nn.Sigmoid()  
        
    def forward(self,x):
    
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size = 3, stride = 1))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size = 3, stride = 3))
        x = F.relu(self.fc1(x.view(-1, 256)))
        #x = self.dropout(x)
        x = self.fc2(x)
        x = self.sigmoid(x) 
        return x # returns a probability 

In [216]:
def train_binary(model, epoch, train_input, train_target, batch_size, 
                 criterion = nn.BCELoss(), optimizer = optim.SGD, 
                 eta = 1e-1, penalty = None, lambda_l2 = 1e-3):  
    # train a binary classifier by minimizing the binary cross entropy loss of a sigmoid output
    # not BCELossWithLogits because sigmoid activation of output already applied in model architecture
    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    train_loss = 0
    for input_, target in (zip(train_input.split(batch_size), train_target.split(batch_size))):
        output = model(input_)
        batch_loss = criterion(output, target.unsqueeze(1))
        if penalty is not None:
            for p in model.parameters():
                batch_loss += lambda_l2 * p.pow(2).sum()
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        train_loss += batch_loss   
    print('Train Epoch: {}  | Loss {:.6f}'.format(
                epoch, train_loss.item()))
    
    

def test_binary(model, test_input, test_target, batch_size, criterion = nn.BCELoss()):
    # test a binary classifier using sigmoid followed by rounding ptobability to 0 or 1
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in zip(test_input.split(batch_size), test_target.split(batch_size)):
            output = model(data)
            batch_loss = criterion(output, target.unsqueeze(1))
            test_loss += batch_loss
            pred = output 
            correct += torch.round(output).squeeze().eq(target).sum() # only need to round because sigmoid already applied in model 
        print('\nTest set:  Loss: {:.4f}, Accuracy: {:.0f}%\n'.format(
        test_loss.item(), 100 * correct/len(test_target)))

In [217]:
###############################
###### Binary Classifier ######
###############################

# initialize
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)
model = Net2C(100)

print(model)

# train
nb_epochs = 50
for epoch in range(1, nb_epochs):
    train_binary(model, epoch, train_input, train_target, 100)

Net2C(
  (conv1): Conv2d(2, 32, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=256, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)
Train Epoch: 1  | Loss 6.857367
Train Epoch: 2  | Loss 6.720750
Train Epoch: 3  | Loss 6.581968
Train Epoch: 4  | Loss 6.427313
Train Epoch: 5  | Loss 6.267084
Train Epoch: 6  | Loss 6.010625
Train Epoch: 7  | Loss 5.759284
Train Epoch: 8  | Loss 5.470505
Train Epoch: 9  | Loss 5.230139
Train Epoch: 10  | Loss 5.042088
Train Epoch: 11  | Loss 4.786299
Train Epoch: 12  | Loss 4.689999
Train Epoch: 13  | Loss 4.686038
Train Epoch: 14  | Loss 4.678190
Train Epoch: 15  | Loss 4.226274
Train Epoch: 16  | Loss 4.188221
Train Epoch: 17  | Loss 3.844506
Train Epoch: 18  | Loss 3.767357
Train Epoch: 19  | Loss 3.518032
Train Epoch: 20  | Loss 3.319799
Train Epoch: 21  | Loss 3.705228
Train Epoch: 22  | Loss 3.383016
Train Ep

In [218]:
# test
test_binary(model, test_input, test_target, 100)


Test set:  Loss: 5.8567, Accuracy: 82%

